In [1]:
import requests, pandas as pd, time
from newspaper import Article
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

BASE_URL = "https://www.hankyung.com/article/"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/127.0.0.0 Safari/537.36"),
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
    "Referer": "https://www.hankyung.com/",
}

def crawl_url(url):
    # 단일 URL 크롤링: requests로 가져와 newspaper 파싱 (403시 /amp 폴백)
    try:
        r = requests.get(url, headers=HEADERS, timeout=12)
        if r.status_code == 403:
            r = requests.get(url.rstrip("/") + "/amp", headers=HEADERS, timeout=12)
        if r.status_code in (403, 404):
            return None
        r.raise_for_status()

        art = Article(url, language='ko')
        art.set_html(r.text)
        art.parse()
        if not art.title or not art.text:
            return None
        return {"url": url, "title": art.title, "text": art.text, "publish_date": art.publish_date}
    except:
        return None


def crawl_i_articles(date_str, max_workers=31):
    """i 기사 크롤링 (0000~9999)"""
    urls = [f"{BASE_URL}{date_str}{num:04}i" for num in range(10000)]
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(crawl_url, u) for u in urls]
        collected = 0
        with tqdm(total=len(futures), desc=f"I {date_str}") as pbar:
            for fut in as_completed(futures):
                res = fut.result()
                if res:
                    results.append(res)
                    collected += 1
                pbar.set_postfix_str(f"수집 {collected}") # 현재 수집 건수
                pbar.update(1)

                time.sleep(0.06)  # 너무 빠른 요청 방지(차단 완화)

    df = pd.DataFrame(results).drop_duplicates(subset=["url"])
    tqdm.write(f"최종 수집 기사 수: {len(df)}") # 최종 수집 건 수
    return df

In [2]:
day_str = "20250820"

In [3]:
out_df = crawl_i_articles(day_str)
out_df.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_i_{day_str}.csv", index=False)

out_df

I 20250820: 100%|██████████| 10000/10000 [11:01<00:00, 15.11it/s, 수집 44]


최종 수집 기사 수: 44


,url,title,text,publish_date
0,https://www.hankyung.com/article/202508200266i,"대통령실, 김여정 담화에 ""남과 북 모두의 안정·번영 위한 것""","""선제적 조치, 일방의 이익이나 누구 의식한 행보 아냐""\n\n""진정성 있는 노력 ...",2025-08-20 09:50:13+09:00
1,https://www.hankyung.com/article/202508200297i,코스피 PBR '10'이라는 부총리…동학개미들 '부글부글' [심성미의 증시 돋보기],"사진=연합뉴스\n\n구윤철 경제부총리 겸 기획재정부 장관이 ""코스피지수의 주가순자산...",2025-08-20 10:19:07+09:00
2,https://www.hankyung.com/article/202508200244i,2027년엔 드론이 밤낮 안 가리고 산불 끄고 공항 새 떼 쫓는다,200kg 싣고 3시간 날아 산불 진화\n\nAI 군집 드론이 활주로 새 떼 막아\...,2025-08-20 11:00:04+09:00
3,https://www.hankyung.com/article/202508200296i,2025 전국수의학도축전 성료… 10개교 수의학도 400여명 참여,"대한민국 수의학 미래들이 한자리\n\n로얄캐닌코리아, 메인 스폰서로 참여\n\n영양...",2025-08-20 10:08:19+09:00
4,https://www.hankyung.com/article/202508200282i,"메타, '인재영입' 수개월 만에 마무리?…조직개편 발표","메타, 'LLM·초지능·제품·인프라' 4개팀으로 개편\n\n\n\n알렉산더 왕이 L...",2025-08-20 10:37:06+09:00
5,https://www.hankyung.com/article/202508200285i,美 압박 틈타 맞손 잡은 중국·인도…5년 만에 국경 무역 재개 합의,2020년 유혈 충돌 이후 단절\n\n비자 완화·수문 정보 공유키로\n\n19일(현...,2025-08-20 10:14:28+09:00
6,https://www.hankyung.com/article/202508200308i,동생 이어 부인까지 '부패 혐의' 수사…퇴진 압박 받는 총리,"부인 고메스, 공금 횡령 혐의로 기소\n\n지난 1년간 측근 줄줄이 부패 수사 받아...",2025-08-20 11:43:12+09:00
7,https://www.hankyung.com/article/202508200346i,"15세기 조선 걸작 청화백자, 홍콩 경매 나온다…추정가 28억~42억",오는 10월 홍콩서 경매\n\n8월 25~27일 서울 전시\n\n이번 경매에 나오는...,2025-08-20 12:01:05+09:00
8,https://www.hankyung.com/article/202508200352i,DL건설 현장서 근로자 추락사…경찰·노동부 압수수색,의정부 신곡동서 추락사고 발생\n\n안전조치 이행 여부 정밀 수사\n\n기사와 무관...,2025-08-20 10:19:46+09:00
9,https://www.hankyung.com/article/202508200379i,"서울시, 비오톱 1등급 규제 완화…""토지 활용 합리성 높인다""",서울시가 비오톱 1등급으로 지정된 땅의 생태적 보전 가치는 유지하면서도 토지이용 현...,2025-08-20 10:27:29+09:00
